In [95]:
#Coluna Testing
using JuMP, BlockDecomposition, Coluna, Gurobi, MathOptInterface, LightGraphs;
const BD = BlockDecomposition;
# const MOI = MathOptInterface;
#Updated on 6/28
#All monitoring decisions are included in the MP
#The coefficients of each arc in the MP is exact
#The path with the least detection probability = most probability of being undetected is approximate

# global gurobi_env = Gurobi.Env()
# setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
# outfile = "C:/Users/din/Documents/GitHub/UncertainTarget/Log_Test_Output.csv"
#     io = open(outfile, "w")
# println(io,"Instance \tPath \tExact \tApprox")
# global p_exact = []
# global p_approx = []

In [2]:
using Pkg
Pkg.status()


    Status `C:\Users\din\.julia\environments\v1.0\Project.toml`
  [6cde8614] BlockDecomposition v1.4.0
  [336ed68f] CSV v0.8.5
  [88b4ec78] Coluna v0.3.10 [`C:\Users\din\.julia\dev\Coluna`]
  [861a8166] Combinatorics v1.0.2
  [a81c6b42] Compose v0.9.2
  [8f4d0f93] Conda v1.5.2
  [a93c6f00] DataFrames v1.2.2
  [31c24e10] Distributions v0.25.16
  [5789e2e9] FileIO v1.11.1
  [a2cc645c] GraphPlot v0.3.1
  [2e9cd046] Gurobi v0.8.1
  [7073ff75] IJulia v1.23.2
  [4076af6c] JuMP v0.21.10
  [86859df6] KnapsackLib v0.4.1 #main (https://github.com/rafaelmartinelli/KnapsackLib.jl)
  [093fc24a] LightGraphs v1.3.5
  [b8f27783] MathOptInterface v0.9.22
  [442fdcdd] Measures v0.3.1
  [eebad327] PkgVersion v0.1.1
  [91a5bcdd] Plots v1.0.14
  [f27b6e38] Polynomials v2.0.14
  [d330b81b] PyPlot v2.10.0
  [1a8c2f83] Query v0.12.2
  [47aef6b3] SimpleWeightedGraphs v1.1.1
  [a759f4b9] TimerOutputs v0.5.12
  [ade2ca70] Dates 
  [de0858da] Printf 


In [4]:
global ins = 1 #:10

include("./TestInstances/Ins"*string(ins)*".jl")
include("./BranchCutPrice_SelfImplementation/functionEnumFeasX.jl")
include("./BranchCutPrice_SelfImplementation/functionFindQ.jl")
include("./BranchCutPrice_SelfImplementation/functionShortestPathBellmanFord.jl")
include("./BranchCutPrice_SelfImplementation/findExactUndetectionPr.jl")
include("./BranchCutPrice_SelfImplementation/functionGenMonitoring.jl")

include("./BranchCutPrice_SelfImplementation/functionSolveNode.jl")

global objVal_inc = 2
global x_inc = []
global y_inc = []

global numArcs = length(arcs[:,1])
global M_set = [[2,6,7]] #EnumX(arcs, b_y, numArcs, d_y)
global numY = 1 #length(M_set)
# println("M_set = ", M_set)
global P_set = [[1,8]]
# println("P_set =", P_set )
global numPaths = 1
global Q = [] #length(Q) = numPaths
global R = 1 
coef_q, Q = findQgivenP(M_set, numY, Q, P_set[1])
coluna = optimizer_with_attributes(
    Coluna.Optimizer,
    "params" => Coluna.Params(
        solver = Coluna.Algorithm.TreeSearchAlgorithm() # default BCP
    ),
    "default_optimizer" => Gurobi.Optimizer # GLPK for the master & the subproblems
)

nb_Monitor = 10
nb_Paths = 1
@axis(M, 1:nb_Monitor)
P = 1:nb_Paths

model = BlockModel(coluna)
@variable(model, x[m in M, j in J], Bin)
@constraint(model, cov[j in J], sum(x[m, j] for m in M) >= 1)
@constraint(model, knp[m in M], sum(w[m, j] * x[m, j] for j in J) <= Q[m])
@objective(model, Min, sum(c[m, j] * x[m, j] for m in M, j in J))
callback_called = false
@dantzig_wolfe_decomposition(model, dwdec, M)
function my_pricing_callback(cbdata)
    # Retrieve the index of the subproblem (it will be one of the values in M)
    cur_SP = BD.callback_spid(cbdata,model)
    red_costs = [BD.callback_reduced_cost(cbdata, x[cur_machine, j]) for j in J]
    current_Obj = JuMP.objective_value.(m)
    y_now = JuMP.value.(y)
    x_now = JuMP.value.(x)
    lambda = zeros(numPaths)
    
    for i = 1:numPaths
        lambda[i] = JuMP.dual(constr[i])
    end
    cur_machine = BD.callback_spid(cbdata, model)

    # Retrieve reduced costs of subproblem variables
    y_red_costs = [BD.callback_reduced_cost(cbdata, y[cur_machine, j]) for j in J]

    # Keep jobs with negative reduced costs
    selected_jobs = Int[]
    selected_costs = Int[]
    for j in J
        if red_costs[j] <= 0.0
            push!(selected_jobs, j)
            # scale reduced costs to int
            push!(selected_costs, -round(Int, 10000 * red_costs[j]))
        end
    end

    if sum(w[cur_machine, selected_jobs]) <= Q[cur_machine]
        # Don't need to run the algorithm,
        # all jobs with negative reduced cost are assigned to the machine
        jobs_assigned_to_cur_machine = selected_jobs
    else
        # Run the algorithm
        items = [KnapItem(w,c) for (w,c) in zip(w[cur_machine, selected_jobs], selected_costs)]
        data = KnapData(Q[cur_machine], items)

        # Solve the knapsack with the algorithm from KnapsackLib
        _, jobs_assigned_to_cur_machine = solveKnapExpCore(data)
    end

    # Create the solution (send only variables with non-zero values)
    sol_vars = [x[cur_machine, j] for j in jobs_assigned_to_cur_machine]
    sol_vals = [1.0 for _ in jobs_assigned_to_cur_machine]
    sol_cost = sum(red_costs[j] for j in jobs_assigned_to_cur_machine)

    # Submit the solution to the subproblem to Coluna
    MOI.submit(model, BD.PricingSolution(cbdata), sol_cost, sol_vars, sol_vals)
    return
end
# my_pricing_callback
# master = BD.getmaster(decomposition)
subproblems = BD.getsubproblems(dwdec)
BD.specify!.(subproblems, lower_multiplicity = 0, solver = my_pricing_callback);
optimize!(model);
println(objective_value(model))

UndefVarError: [91mUndefVarError: J not defined[39m

In [201]:

function get_dual_of_generated_cuts()
    
    ins = 1
    include("./TestInstances/Ins"*string(ins)*".jl")
    include("./BranchCutPrice_SelfImplementation/functionEnumFeasX.jl")
    include("./BranchCutPrice_SelfImplementation/functionFindQ.jl")
    include("./BranchCutPrice_SelfImplementation/functionShortestPathBellmanFord.jl")
    include("./BranchCutPrice_SelfImplementation/findExactUndetectionPr.jl")
    include("./BranchCutPrice_SelfImplementation/functionGenMonitoring.jl")

#     include("./functionSolveNode.jl")
    coluna = JuMP.optimizer_with_attributes(
        Coluna.Optimizer,
        "params" => Coluna.Params(
            solver=Coluna.Algorithm.TreeSearchAlgorithm(),
        ),
        "default_optimizer" => Gurobi.Optimizer 
    );
#     println(arcs)
    global P_set = P
    println(P_set)
    global M_set = [[1,7]] 
    global numY = length(M_set)
    println(M_set)
    global numArcs = length(arcs[:,1])
    global numPaths = length(P_set)
    global Q = [] #length(Q) = numPaths
    global R = 1 
    # coef_q, Q = findQgivenP(M_set, numY, Q, P_set[1])
    for path in P_set
        global Q
        coef_q, Q = findQgivenP(M_set, numY, Q, path)
    end
    println(Q)
#     cons = Array{JuMP.ConstraintRef}(undef, 2)
#     lazyCon = Coluna.MathProg.Id{Coluna.MathProg.Constraint}
#     println(typeof(constrid))
    model = BlockModel(coluna, direct_model=true)

    @axis(I, 1:1)
#     @variable(model, 0 <= z[i in I] <= 1)
    @variable(model, x[i in I, 1:numArcs], Bin) # subproblem variables & constraints
    @variable(model, y[i in I, 1:numArcs] >= 0) # master
    @variable(model, u[i in I] >=0) # master
    @variable(model, 0<=w<=1)
    
    @constraint(model, xCon[i in I], sum(d_x[a]*x[i, a] for a = 1:numArcs) <= b_x)
    @constraint(model, yCon[i in I], sum(y[i, m] for m = 1:numY) == 1)
#     @constraint(model, uCon[i in I], u[i] >= 0.9*y[i,1] + y[i,2] - x[i,1] - x[i,2] - x[i,3])
    @constraint(model, wCon, w >= sum(x[i,a] for i in I, a =1:numArcs))
    println("A")
#     for path = 1:numPaths
    @constraint(model, uCon1[i in I], u[i] >= sum(y[i,m]*Q[1][m] for m = 1:numY) - R*(sum(x[i,a] for a in P_set[1])))
    @constraint(model, uCon2[i in I], u[i] >= sum(y[i,m]*Q[2][m] for m = 1:numY) - R*(sum(x[i,a] for a in P_set[2])))
    @constraint(model, uCon3[i in I], u[i] >= sum(y[i,m]*Q[3][m] for m = 1:numY) - R*(sum(x[i,a] for a in P_set[3])))
    @constraint(model, uCon4[i in I], u[i] >= sum(y[i,m]*Q[4][m] for m = 1:numY) - R*(sum(x[i,a] for a in P_set[4])))
#     @constraint(model, uCon5[i in I], u[i] >= sum(y[i,m]*Q[5][m] for m = 1:numY) - R*(sum(x[i,a] for a in P_set[5])))
    @objective(model, Min, w)
    
#     cons[1] = @constraint(model, initCon1, u >= 0.9*y[1] + y[2] - x[1] - x[2] - x[3])
#     cons[2] = @constraint(model, initCon2, u >= y[1] + y[2] - x[7])

    println("B")

    callback_called = false
    constrid = nothing
    #constrid = Coluna.MathProg.Id{Coluna.MathProg.Constraint}(undef, 1)
    #nothing
    
    function my_callback_function(cbdata)
#         global constrid
        if !callback_called
            println("Called")
#             i = BD.callback_spid(cbdata, model)
#             con = @build_constraint(w >= u[1])
            con = @build_constraint(w >= sum(y[1,m]*Q[5][m] for m = 1:numY) - R*(sum(x[1,a] for a in P_set[5])))
            constrid = MOI.submit(model, MOI.LazyConstraint(cbdata), con)
#             println(typeof(constrid_))
            callback_called = true
        end
        return
    end
    
    function my_pricing_callback(cbdata)
        # Retrieve the index of the subproblem (it will be one of the values in M)
#         cur_machine = BD.callback_spid(cbdata, model)
        global constrid #, lambda1
        @show constrid
        cur_i = BD.callback_spid(cbdata, model)

        # Retrieve reduced costs of subproblem variables
        red_costs = [BD.callback_reduced_cost(cbdata, y[cur_i, j]) for j=1:numArcs]
        println("REDUCED COSTS = ", red_costs)
#         println("lambda1 = ", lambda1)
        #getcurdual
#         M = genMonitoring(lambda, P_set)
        # Create the solution (send only variables with non-zero values)
        
#         sol_vars = [y[cur_i, j] for j in jobs_assigned_to_cur_machine]
#         sol_vals = [1.0 for _ in jobs_assigned_to_cur_machine]
#         sol_cost = sum(red_costs[j] for j in jobs_assigned_to_cur_machine)
        jobs_assigned_to_cur_machine = [2]
        sol_vars = [y[cur_i, j] for j in jobs_assigned_to_cur_machine] #y[cur_i, 2] 
        sol_vals = [-0.5 for _ in jobs_assigned_to_cur_machine]
        sol_cost = [-1000 for _ in jobs_assigned_to_cur_machine] #sum(red_costs[j] for j in jobs_assigned_to_cur_machine)
#         println("sol_vars = ", sol_vars)
#         println("sol_vals = ", sol_vals)
#         println("sol_cost = ", sol_cost)
        
        # Submit the solution to the subproblem to Coluna
        MOI.submit(model, BD.PricingSolution(cbdata), sol_vars, sol_vals)
        println("DONE HERE")
    end
    
    
    @dantzig_wolfe_decomposition(model, dec, I)
    subproblems = BD.getsubproblems(dec);
    BD.specify!.(subproblems, lower_multiplicity = 0, solver = my_pricing_callback);
    
    MOI.set(model, MOI.LazyConstraintCallback(), my_callback_function)
   
#     lambda1 = MOI.get(JuMP.unsafe_backend(model), MOI.ConstraintDual(), constrid) #BD.callback_dual_sols(cbdata,uCon1[cur_i])
#         lambda1 = BD.getcurdual(cbdata,uCon1[cur_i])
#         lambda1 = MOI.get(JuMP.unsafe_backend(cbdata), MOI.ConstraintDual(), uCon1[cur_i])
#     
   
#     println(model)
    optimize!(model)
    lambda1 = MOI.get(JuMP.unsafe_backend(model), MOI.ConstraintDual(), constrid)
    println("LAMBDA = ", lambda1)
#     println(model)
#     println("u = ", objective_value(model))
    
#     println(MOI.ConstraintDual(cons[1]))
#     println(MOI.get(JuMP.unsafe_backend(model), MOI.ConstraintDual(), constrid[2]))
#     println(MOI.get(JuMP.unsafe_backend(model), MOI.ConstraintDual(), constrid))
#     @test objective_value(model) ≈ 0.63333333
#     @test MOI.get(JuMP.unsafe_backend(model), MOI.ConstraintDual(), constrid) ≈ 0.33333333
    
#     for i = 1:1
#         Z = value.(x[i,:]).data  # j-th position is equal to 1 if job j assigned to machine i
#         println("x_", i,": ", findall(Z.>0))
#         Z = value.(y[i,:]).data  # j-th position is equal to 1 if job j assigned to machine i
#         println("y_", i,": ", findall(Z.>0))
#     end
    
    return
end

get_dual_of_generated_cuts (generic function with 1 method)

In [202]:
get_dual_of_generated_cuts()

Array{Int64,1}[[1, 8], [2, 9], [3], [4, 6], [5, 7]]
Array{Int64,1}[[1, 7]]
Any[[0.2], [1.0], [1.0], [1.0], [0.9]]
A
B
Coluna
Version 0.3.10 | 2021-09-07 | https://github.com/atoptima/Coluna.jl
***************************************************************************************
**** BaB tree root node
**** Local DB = -Inf, global bounds : [ -Inf , Inf ], time = 0.00 sec.
***************************************************************************************
Called
Robust cut separation callback adds 1 new essential cuts and 0 new facultative cuts.
avg. viol. = -0.00, max. viol. = -0.00, zero viol. = 1.
constrid = nothing
REDUCED COSTS = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


ErrorException: [91mtype Array has no field func[39m